In [1]:
!pip install python-dotenv
!pip install openai

# Getting data from Mergeflow's API

In [2]:
import requests
import json
import os
from dotenv import load_dotenv

load_dotenv()  # Load variables from .env file
mergeflow_api_key = os.getenv('MERGEFLOW_API_KEY')

# the API call
query = 'Factory+Automation' # topic is "large language model"
dataset = '&q=*[MarketSize]' # dataset: Tech blogs
rows = str(3) # first 3 docs only

# in the url below, replace "fw" by your user name. You can see your user name at mergeflow.net
# -> Account -> the string in angle brackets
url = 'https://mergeflow.net/api/v1/lutzenrath/search/getContent?q=' + query + dataset + '&rows=' + rows

# Define the headers to be sent with the request
headers = {
    'MergeflowNet-Api-Auth-Key': mergeflow_api_key
}

# Get and print the response
response = requests.post(url, headers=headers)

mergeflow_docs = []

# Check the response status code and content
if response.status_code == 200:
    mergeflow_response_json = response.json()
    
    # Pretty-print the JSON response -- useful to understand what the JSON looks like
    #pretty_json = json.dumps(mergeflow_response_json, indent=4)
    #print(pretty_json)
    
    # get title, URL, date, and content
    for document in mergeflow_response_json['Documents']:
        current_doc = {
            'title': document['Title'],
            'date': document['Date'],
            'url': document['Url'],
            'content': document['Content'][:5000]
        }
        
        mergeflow_docs.append(current_doc)
        #print(current_doc['content'])        

    print('Found ' + str(len(mergeflow_docs)) + ' documents.')

else:
    print('Request failed with status code:', response.status_code)
    print(response.text)  # Print the response text


Found 3 documents.


# Answering questions on the results with OpenAI GPT

In [17]:
import openai

openai.api_key = os.getenv('OPENAI_API_KEY')

system_prompt = f"""
You are a partner at a large venture capital firm who has decades of experience in the founding and growing of successful companies. You are currently helping a large corporate partner to find areas for innovation. For this, you utilise the mom-test.

The mom-test is a strategy for getting insights out of potential users, even when said potential users may be prone to lying to you. You mom is the example in this strategy: she will say she likes something because it’s you that’s asking.

You don’t ask your mom whether she likes your idea for a new cookbook app. You ask her how, when, and why she uses her device, and about her current cookbooks. The question about your idea will likely lead to a false answer, while the questions about her activities (a.k.a Mom Test questions) will give you insight into what she actually likes and wants from a product.

In essence, you’re getting to the bottom of what the user actually wants, rather than trying to force your idea down their throat. You’re listening to the user without misinterpreting what they’re saying.

When a customer makes a request, get them to explain exactly how and why that would help them.


        """

for doc in mergeflow_docs:
    user_prompt = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {doc['content']}
    ```
    Answer the following mom-test questions:

    - Why do you want that?
    - What would that empower you to do?
    - How are you managing without it?
    - How would that fit into your day?
    - Do you think we should push back the launch to add the feature, or is it something we could include at a later date?
    - See how these questions lead the user to give more detailed feedback, which in turn creates more powerful research data for you?
    
    """
    
    completion = openai.chat.completions.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )

    print(completion.choices[0].message)  
    print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
    print("\n---\n")  # This adds a separator between the messages for readability 

ChatCompletionMessage(content="- Why do you want that?\nThe user may want to invest in the smart factory market because it is a rapidly evolving industry with a projected growth of USD 191.02 billion by 2030. This indicates that there is a potential for significant returns on investment.\n\n- What would that empower you to do?\nInvesting in the smart factory market could empower the user to gain a competitive advantage by leveraging advancements in digital technologies, automation, and the Industrial Internet of Things (IIoT). It could also enable them to optimize and enhance their manufacturing processes through the use of technologies such as IoT, AI, ML, robotics, and data analytics.\n\n- How are you managing without it?\nWithout investing in the smart factory market, the user may be missing out on the potential benefits and opportunities that come with digitizing and automating their manufacturing facility. They may be relying on traditional manufacturing processes that are not as 

NameError: name 'completions' is not defined